In [2]:
import itertools
import random
from pprint import pprint
from natsort import natsorted

random.seed(42)

# 定义G、T、R的范围
G_range = [1, 2, 3, 4]
T_range = [1, 2, 3, 4, 5]
R_range = [1, 2, 3, 4, 5, 6]

whisper_combinations = [(0,t,0) for t in T_range]

# 生成所有可能的GTR组合
all_combinations = list(itertools.product(G_range, T_range, R_range))
# 重新组织代码，以按G组、T组、R组分类trials，然后再进行sub-section分配

# 初始化用于存储按G、T、R组分类的trials的字典
trials_by_g = []
trials_by_t = []
trials_by_r = []

# 在一个trial中，包括reference在内，最多给参与者听4个音频
# 因此，对于每个参考样式，我们将选择最多2个变化探针

def select_variants(variants, limit=2):
    # 如果变化探针数量超过限制，则随机选择
    if len(variants) > limit:
        return random.sample(variants, limit)
    return variants

# 生成trials的函数，以选择限定数量的变化探针
def trials_for_dimension(dimension, limit=2):
    trials = []
    for g, t, r in all_combinations:
        reference = f'{g}{t}{r}'
        correct_probe = reference
        if dimension == 'G':
            variant_probes = select_variants([f'{g_var}{t}{r}' for g_var in G_range if g_var != g], limit)
        elif dimension == 'T':
            variant_probes = select_variants([f'{g}{t_var}{r}' for t_var in T_range if t_var != t], limit)
        else: # dimesion == 'R'
            variant_probes = select_variants([f'{g}{t}{r_var}' for r_var in R_range if r_var != r], limit)
        trials.append({
            'dimension': dimension,
            'reference': reference, 
            'correct_probe': correct_probe, 
            'variant_probes': variant_probes
        })
    return trials

def generate_whisper_combinations_trials(T_range, limit=2):
    whisper_trials = []
    for t in T_range:
        reference = f'0{t}0'
        correct_probe = reference
        # 对于T维度，variant_probes从其他T值中选择，但G和R保持为0
        variant_probes = select_variants([f'0{t_var}0' for t_var in T_range if t_var != t], limit)
        whisper_trials.append({
            'dimension': 'T',  # 特殊组合主要用于探查T维度
            'reference': reference,
            'correct_probe': correct_probe,
            'variant_probes': variant_probes
        })
    return whisper_trials

whisper_combinations_trials = generate_whisper_combinations_trials(T_range)

# 生成各维度的trials
trials_g = trials_for_dimension('G')
trials_t = trials_for_dimension('T')
trials_r = trials_for_dimension('R')
num_sub_sections = 1

# 合并所有trials并随机分配到sub-sections
all_trials_combined = trials_g + trials_t + trials_r 
all_trials_combined += whisper_combinations_trials

# 随机打乱trials顺序
random.shuffle(all_trials_combined) 

sub_sections_combined = [all_trials_combined[i::num_sub_sections] for i in range(num_sub_sections)]

# 打印每个sub-section的trials，标记评估的维度
for i, sub_section in enumerate(sub_sections_combined, start=1):
    print(f"Sub-section {i}:")
    for trial in sub_section:
        print(f"Dimension: {trial['dimension']}, Reference: {trial['reference']}, Correct Probe: {trial['correct_probe']}, Variant Probes: {trial['variant_probes']}")
    print("\n")

# natsorted(sub_sections_combined)


# len of each sub-section
print([len(sub_section) for sub_section in sub_sections_combined])


# len of all trials
print(len(all_trials_combined))


Sub-section 1:
Dimension: T, Reference: 351, Correct Probe: 351, Variant Probes: ['321', '311']
Dimension: T, Reference: 314, Correct Probe: 314, Variant Probes: ['324', '334']
Dimension: R, Reference: 121, Correct Probe: 121, Variant Probes: ['123', '126']
Dimension: R, Reference: 133, Correct Probe: 133, Variant Probes: ['132', '131']
Dimension: R, Reference: 341, Correct Probe: 341, Variant Probes: ['342', '346']
Dimension: G, Reference: 421, Correct Probe: 421, Variant Probes: ['121', '321']
Dimension: T, Reference: 246, Correct Probe: 246, Variant Probes: ['216', '256']
Dimension: R, Reference: 143, Correct Probe: 143, Variant Probes: ['142', '145']
Dimension: R, Reference: 325, Correct Probe: 325, Variant Probes: ['326', '322']
Dimension: R, Reference: 333, Correct Probe: 333, Variant Probes: ['335', '334']
Dimension: G, Reference: 213, Correct Probe: 213, Variant Probes: ['413', '113']
Dimension: T, Reference: 145, Correct Probe: 145, Variant Probes: ['115', '135']
Dimension: T,

In [9]:
all_trials_combined[0]

{'dimension': 'T',
 'reference': '351',
 'correct_probe': '351',
 'variant_probes': ['321', '311']}

In [3]:
import os

# initialize trials mapping
trials_mapping = {}

# base path
base_path = "/storageNVME/kcriss/gtr_dataset_evaluation/"

# iterate through all trials and create the directory structure
for i, trial in enumerate(all_trials_combined, start=1):
    dimension = trial['dimension']
    reference = trial['reference']
    correct_probe = trial['correct_probe']
    variant_probes = trial['variant_probes']

    # create paths
    dimension_path = os.path.join(base_path, dimension)
    reference_path = os.path.join(dimension_path, 'reference', reference)
    correct_probe_path = os.path.join(dimension_path, 'correct_probe', correct_probe)
    variant_probe_paths = [os.path.join(dimension_path, f'variant_probe_{idx+1}', vp) for idx, vp in enumerate(variant_probes)]

    # # create directories
    # os.makedirs(reference_path, exist_ok=True)
    # os.makedirs(correct_probe_path, exist_ok=True)
    # for vp_path in variant_probe_paths:
    #     os.makedirs(vp_path, exist_ok=True)
        
    # 更新映射表的键，使用维度和参考的组合
    key = f"Trail_{i}_{dimension}_{reference}"

    # 映射trial信息
    trials_mapping[key] = {
        'dimension': dimension,
        'reference_path': reference_path,
        'correct_probe_path': correct_probe_path,
        'variant_probe_paths': variant_probe_paths
    }

# 示范如何访问映射表中的信息
for key, info in trials_mapping.items():
    dimension, reference = key.split('_', 1)  # 分割键以获取维度和参考
    print(f"Key: {key}")
    print(f"Dimension: {dimension}, Reference: {reference}")
    print(f"Reference Path: {info['reference_path']}")
    print(f"Correct Probe Path: {info['correct_probe_path']}")
    for idx, vp_path in enumerate(info['variant_probe_paths'], start=1):
        print(f"Variant Probe {idx} Path: {vp_path}")
    print("\n")

Key: Trail_1_T_351
Dimension: Trail, Reference: 1_T_351
Reference Path: /storageNVME/kcriss/gtr_dataset_evaluation/T/reference/351
Correct Probe Path: /storageNVME/kcriss/gtr_dataset_evaluation/T/correct_probe/351
Variant Probe 1 Path: /storageNVME/kcriss/gtr_dataset_evaluation/T/variant_probe_1/321
Variant Probe 2 Path: /storageNVME/kcriss/gtr_dataset_evaluation/T/variant_probe_2/311


Key: Trail_2_T_314
Dimension: Trail, Reference: 2_T_314
Reference Path: /storageNVME/kcriss/gtr_dataset_evaluation/T/reference/314
Correct Probe Path: /storageNVME/kcriss/gtr_dataset_evaluation/T/correct_probe/314
Variant Probe 1 Path: /storageNVME/kcriss/gtr_dataset_evaluation/T/variant_probe_1/324
Variant Probe 2 Path: /storageNVME/kcriss/gtr_dataset_evaluation/T/variant_probe_2/334


Key: Trail_3_R_121
Dimension: Trail, Reference: 3_R_121
Reference Path: /storageNVME/kcriss/gtr_dataset_evaluation/R/reference/121
Correct Probe Path: /storageNVME/kcriss/gtr_dataset_evaluation/R/correct_probe/121
Varian

In [4]:

# save trials_mapping to a list
trials_mapping_list = []
for reference, info in trials_mapping.items():
    trials_mapping_list.append({
        'reference': reference,
        'dimension': info['dimension'],
        'reference_path': info['reference_path'],
        'correct_probe_path': info['correct_probe_path'],
        'variant_probe_paths': info['variant_probe_paths']
    })


In [75]:
import os
import shutil


# 源目录路径，其中存放了.wav音频文件
source_dir = '/storageNVME/kcriss/gtr_dataset_sliced_FREEZE/gtr_125'

# 遍历trials_mapping_list中的每个条目
for trial in trials_mapping_list:
    # 处理correct_probe
    correct_probe_gtr_combination = trial['reference_path'].split('/')[-1]
    correct_probe_source_dir = os.path.join(source_dir, correct_probe_gtr_combination)
    if os.path.exists(correct_probe_source_dir):
        for i in range(1, 21):  
            src_file = os.path.join(correct_probe_source_dir, f'{i:02}.wav')
            if os.path.exists(src_file):
                shutil.copy(src_file, trial['reference_path'])
    else:
        print(f"Source directory does not exist: {correct_probe_source_dir}")


    # for variant_path in trial['variant_probe_paths']:
    #     variant_gtr_combination = variant_path.split('/')[-1]
    #     variant_source_dir = os.path.join(source_dir, variant_gtr_combination)
    #     if os.path.exists(variant_source_dir):
    #         for i in range(1, 21): 
    #             src_file = os.path.join(variant_source_dir, f'{i:02}.wav')
    #             if os.path.exists(src_file):
    #                 shutil.copy(src_file, variant_path)
    #     else:
    #         print(f"Source directory does not exist: {variant_source_dir}")


In [ ]:
# save list to a json file
import json
with open('trials_mapping.json', 'w') as f:
    json.dump(trials_mapping_list, f, indent=4)

In [5]:

html_fixed_head_content = '''
<!DOCTYPE html>
<html lang="en">
<head>
<style>
  table audio {
          width: 100%;
          max-width: 300px; 
  }
  table {
          width: 100%;
          border-collapse: collapse;
      }

      table, th, td {
          border: 1px solid black;
      }

      th, td {
          padding: 10px;
          text-align: left;
      }
</style>

<body>

<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src=""></script>
<script>
  window.dataLayer = window.dataLayer || [];

  function gtag() {
    dataLayer.push(arguments);
  }

  gtag('js', new Date());

  gtag('config', '');
</script>

<section>
  <div>

    
    <div>
      <div>
        <h1>Subjective Evaluation (主观评估)</h1>
        <p>Thank you for participating in the subjective evaluation. (感谢您参与本主观评估测试。)</p>
          <p><strong>Instructions (测试说明)</strong>: </p>
            <p>1. You will hear 18~19 groups of voices, each group contains a reference voice and three test voices. (你将听到18~19组语音，每组语音包含一个参考语音和三个测试语音。)</p>
            <p>2. The text content of the reference voice is different from the text content of the test voice. (参考语音的文本内容与测试语音的文本内容不同。)</p>
            <p>3. Click the triangle button on the left side of the audio block with the mouse to play the audio. (用鼠标点击音频块左侧的三角形按钮播放音频。)</p>
            <p>4. You need to choose one of the three test voices that is most similar to the reference voice, and click the corresponding round single selection button with the mouse. (你需要在三个测试语音中选择一个与参考最相似的语音，并用鼠标点击相应的圆形单选按钮。)</p>
          <p><strong>Most similar might mean (最相似可能意味着):</strong> </p>
          <ul>
            <li><strong>The pitch/timbre of the voice is similar (语音的音高/音色相似)</strong></li>
            <li><strong>The style/speed/rhythm of the pronunciation of the voice is similar (语音的吐字或发声方式以及风格相似)</strong></li>
          </ul>
          <br>
    '''

In [6]:
html_fixed_tail_content = '''


    <br><br>

        <!-- Submit button -->
            <div style="text-align: center;">
                <input type="submit" value="Complete" style="width: 150px; height: 50px;">
            </div>
            </form>

            <div id="resultText" style="text-align: center; display: none;">
                <p>Please copy the following text and paste it into Google Forms(请复制以下文本并粘贴到Google Forms中):</p>
                <textarea id="resultOutput"></textarea>
            </div>
            <div style="display: flex; justify-content: center; margin-top: 20px;"> <!-- 开始一个新的 div 用于 iframe -->
                <iframe src="https://docs.google.com/forms/d/e/1FAIpQLSdJdZbJIQPeFQHEHIbMKYjmjL0gGPGhx08AY8G-hoFcD3QeSA/viewform?embedded=true" width="640" height="451" frameborder="0" marginheight="0" marginwidth="0">Loading…</iframe>
            </div>
            </section>
        </div>
  <script>
  document.getElementById('evaluationForm').onsubmit = function(event) {
      event.preventDefault(); // 阻止表单默认提交行为

      var resultArray = [];
      var currentTime = new Date().toISOString();
      var evaluationIdx = document.getElementById('evaluationIdx').value; // 从隐藏的输入获取evaluation_idx
      var evaluationNumber = "evaluation_" + evaluationIdx; // 使用evaluation_idx构建评估编号

      resultArray.push(evaluationNumber);

      // 获取每个group的用户选择
      var choices = [];
      for (var i = 1; i <= 18; i++) {
          var groupName = "group_" + i;
          var selectedValue = document.querySelector('input[name="' + groupName + '"]:checked')?.value;
          choices.push(selectedValue || "N/A");
      }

      resultArray.push(choices.join(","));

      // 显示结果文本并填充到textarea中
      var resultText = resultArray.join(",");
      document.getElementById('resultOutput').textContent = resultText;
      document.getElementById('resultText').style.display = 'block';
  };
  </script>
          var resultText = resultArray.join(",");
          var resultOutput = document.getElementById('resultOutput');
          resultOutput.textContent = resultText;
          resultOutput.style.height = 'auto';
          resultOutput.style.height = (resultOutput.scrollHeight) + 'px';
          document.getElementById('resultText').style.display = 'block';
</body>
</html>
'''

In [10]:
# generate html  
import random

def generate_groups_html(trials_mapping_list, start_index, end_index, evaluation_idx=1):
    html_output = ""
    
    form_html_start = '<form id="evaluationForm">'
    hidden_input_for_evaluation_idx = f'<input type="hidden" id="evaluationIdx" value="{evaluation_idx}">'
    form_html = form_html_start + hidden_input_for_evaluation_idx

    server_root_prefix = "/storageNVME/kcriss/gtr_dataset_evaluation/"

    for i in range(start_index, end_index):
        trial = trials_mapping_list[i]
        # 将绝对路径转换为相对于服务器根目录的相对路径
        reference_audio_rel_path = trial['reference_path'].replace(server_root_prefix, "")
        correct_probe_audio_rel_path = trial['correct_probe_path'].replace(server_root_prefix, "")
        variant_probe_audios_rel_paths = [vp.replace(server_root_prefix, "") for vp in trial['variant_probe_paths']]

        # 随机选择一个.wav文件编号
        selected_file_number = f"{random.randint(1, 20):02}.wav"

        # 生成音频标签，引用随机选择的相同编号的.wav文件
        reference_tag = f'<td style="text-align: center"><audio src="{reference_audio_rel_path}/{selected_file_number}" style="max-width: 300px;" type="audio/wav" controls controlsList="nodownload noplaybackrate"></audio></td>'
        correct_probe_tag = f'<td style="text-align: center"><audio src="{correct_probe_audio_rel_path}/{selected_file_number}" style="max-width: 300px;" type="audio/wav" controls controlsList="nodownload noplaybackrate"></audio></td>'
        variant_probe_tags = ''.join([f'<td style="text-align: center"><audio src="{vp_rel_path}/{selected_file_number}" style="max-width: 300px;" type="audio/wav" controls controlsList="nodownload noplaybackrate"></audio></td>' for vp_rel_path in variant_probe_audios_rel_paths])  
        
        # 生成单选按钮
        radio_buttons = ''.join([
                f'<td style="text-align: center;"><input type="radio" id="group_{i+1}_choice_{j+1}" name="group_{i+1}" value="{j+1}" required><label for="group_{i+1}_choice_{j+1}">This one is most similar to reference</label></td>'
                for j in range(3)
        ])

        # 拼接完整的HTML代码
        group_html = f'''

        <!-- Group {i+1-start_index} -->
        <h3>Group {i+1-start_index}</h3>
        <div>
        </div>
        <div><table style="table-layout:fixed;">
            <col span="1" style="width: 25%;">
            <col span="1" style="width: 25%;">
            <col span="1" style="width: 25%;">
            <col span="1" style="width: 25%;">
            <thead>
                <tr>
                    <th style="text-align: center">Reference</th>
                    <th style="text-align: center">Test Sample 1</th>
                    <th style="text-align: center">Test Sample 2</th>
                    <th style="text-align: center">Test Sample 3</th>
                </tr>
            </thead>
            <tbody>
                <tr>{reference_tag}{correct_probe_tag}{variant_probe_tags}</tr>
                <tr>
                <td>&#128204; Which speech sample(Sample 1, Sample 2, Sample 3) sounds most similar to the reference?</td>
                {radio_buttons}
                </tr>
            </tbody>
        </table></div>
        <!-- End of Group {i+1-start_index}. -->
        
        '''
        
  
        html_output += group_html + "\n\n"

    html_output = html_fixed_head_content + form_html + html_output + html_fixed_tail_content

    return html_output


In [11]:
# 生成第一个HTML文档的代码，包含第1到第18组
html_content_1 = generate_groups_html(trials_mapping_list, 0, 18, 1)
# save to file
with open('evaluation_1.html', 'w') as f:
    f.write(html_content_1)

# # 生成第二个HTML文档的代码，包含第19到第37组
# html_content_2 = generate_groups_html(trials_mapping_list, 18, 37)
# # save to file
# with open('evaluation_2.html', 'w') as f:
#     f.write(html_content_2)

In [9]:
# show root working directory
!pwd

/storageNVME/kcriss/gtr_dataset_evaluation
